# Tarea 2 - Sistema de recomendación
## Inteligencia de negocios

### Bibliotecas usadas

In [1]:
import pandas as pd
from dotenv import load_dotenv
import requests
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy import sparse
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

plt.rcParams["figure.figsize"] = (12,10)
load_dotenv()

C:\Users\Melissa\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


True

In [25]:
movie_titles = pd.read_csv("movie_titles.csv", usecols=range(3), names=['id', 'year', 'name'], encoding='ISO-8859-1')
movie_titles

,id,year,name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


#### Identificación y corrección de columnas vacías

In [26]:
print("----- Campos NaN en columna id -----\n", movie_titles[movie_titles['name'].isna()])
print("----- Campos NaN en columna año -----\n", movie_titles[movie_titles['year'].isna()])
print("----- Campos NaN en columna nombre -----\n", movie_titles[movie_titles['id'].isna()])

----- Campos NaN en columna id -----
 Empty DataFrame
Columns: [id, year, name]
Index: []
----- Campos NaN en columna año -----
           id  year                                         name
4387    4388   NaN      Ancient Civilizations: Rome and Pompeii
4793    4794   NaN  Ancient Civilizations: Land of the Pharaohs
7240    7241   NaN     Ancient Civilizations: Athens and Greece
10781  10782   NaN                       Roti Kapada Aur Makaan
15917  15918   NaN                      Hote Hote Pyaar Ho Gaya
16677  16678   NaN                              Jimmy Hollywood
17666  17667   NaN                           Eros Dance Dhamaka
----- Campos NaN en columna nombre -----
 Empty DataFrame
Columns: [id, year, name]
Index: []


#### Llenado de la información faltante
Para no tener que eliminar estas películas, primero probamos la API de The Movie Database, en caso de que esta pudiera entregarnos los años de cada película.

In [27]:
api_key = os.getenv("API_KEY")

for movie in movie_titles[movie_titles['year'].isna()]['name']:
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie}"
    response = requests.get(url)
    data = response.json()
    try:
        relase_date = data['results'][0]['release_date']
        if relase_date == '':
            raise
        release_year = relase_date[:4]
        print("🟢 La fecha de lanzamiento de " + movie + " es " + relase_date + 
              ", del año " + release_year + ".")
        movie_titles.loc[movie_titles['name'] == movie, 'year'] = release_year
    except: print("🔴 No se encontró fecha de lanzamiento para " + movie + ".")

🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Rome and Pompeii.
🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Land of the Pharaohs.
🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Athens and Greece.
🟢 La fecha de lanzamiento de Roti Kapada Aur Makaan es 1974-01-01, del año 1974.
🔴 No se encontró fecha de lanzamiento para Hote Hote Pyaar Ho Gaya.
🟢 La fecha de lanzamiento de Jimmy Hollywood es 1994-03-30, del año 1994.
🔴 No se encontró fecha de lanzamiento para Eros Dance Dhamaka.


Ya que solo se encontró el año de lanzamiento de dos películas, se procede a hacer una búsqueda manual en Google para llenar las películas faltantes.

In [28]:
movie_titles.at[4387, 'year'] = 2001
movie_titles.at[4793, 'year'] = 2001
movie_titles.at[7240, 'year'] = 2002
movie_titles.at[15917, 'year'] = 1999
movie_titles.at[17666, 'year'] = 1999
movie_titles = movie_titles.astype(dtype = {'id': np.int32, 'year': np.int32})
print("----- Campos NaN en columna año -----\n", movie_titles[movie_titles['year'].isna()])

----- Campos NaN en columna año -----
 Empty DataFrame
Columns: [id, year, name]
Index: []


#### Lectura de las calificaciones para cada película

In [7]:
'''
ratings_folder = 'c:\\umayor\\training_set'
files = ['\\mv_{}.txt'.format(str(f).zfill(7)) for f in range(1,17771)]
df_list = []

for filename in tqdm(files):
    temp = pd.read_csv(ratings_folder + filename, names = ['user', 'rating', 'year'], skiprows = 1).astype(dtype = {'user': np.single, 'rating': np.single}).drop(columns = ['year'])
    temp['movie'] = filename.split('.')[0].split('_')[1].lstrip('0')

    interactions = (temp.groupby(['user', 'movie'])['rating']
      .sum()
      .unstack()
      #.reset_index()
      #.fillna(0)
      #.set_index('user')
    )
    #print(interactions)
    df_list.append(interactions)
'''

100%|██████████| 17770/17770 [02:29<00:00, 118.56it/s]


#### Almacenamiento en una matriz pivote

In [8]:
'''
pivot_df = pd.DataFrame()
pivot_df = pd.concat(df_list, axis=1)
pivot_df = pivot_df.fillna(0)
pivot_df
'''

movie,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
'''
pivot_df.to_parquet('ratings_df.parquet')
'''

In [2]:
import fastparquet

In [3]:
pivot_df = pd.read_parquet('ratings_df.parquet', engine='fastparquet')

In [4]:
pivot_df

movie,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Generación de matriz dispersa con compresión de los 0 por fila

In [5]:
sparse_matrix = sparse.csr_matrix(pivot_df.to_numpy())
del pivot_df
train_test = random_train_test_split(sparse_matrix, test_percentage=0.2)
train = train_test[0]
test = train_test[1]

#### Selección del modelo y entrenamiento

In [6]:
model = LightFM(loss='warp')
model.fit(train, epochs=5, verbose=True)
with open('model.pickle', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)

Epoch: 100%|██████████| 5/5 [12:50<00:00, 154.18s/it]


https://stackoverflow.com/questions/45451161/evaluating-the-lightfm-recommendation-model/45466481#45466481

In [7]:
train_precision = precision_at_k(model, train, k=5).mean()
test_precision = precision_at_k(model, train, test, k=5).mean()
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

In [9]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.47, test 0.55.
AUC: train 0.97, test 0.96.


In [17]:
suggested_movies = model.predict(915, [x for x in range(1,17770)])
suggested_movies

array([-2.687106  , -2.6217592 , -2.3154707 , ..., -1.8072433 ,
       -0.43457633, -1.7180575 ], dtype=float32)

In [2]:
suggested_movies.sort()

NameError: name 'suggested_movies' is not defined

In [4]:
'''
ratings_folder = 'c:\\umayor\\training_set'
files = ['\\mv_{}.txt'.format(str(f).zfill(7)) for f in range(1,17771)]
df_list_binary = []

for filename in tqdm(files):
    temp = pd.read_csv(ratings_folder + filename, names = ['user', 'rating', 'year'], skiprows = 1)
    temp['rating'].values[temp['rating'].values < 4] = -1
    temp['rating'].values[temp['rating'].values >= 4] = 1
    temp = temp.astype(dtype = {'user': np.single, 'rating': np.single}).drop(columns = ['year'])
    temp['movie'] = filename.split('.')[0].split('_')[1].lstrip('0')
    interactions = (temp.groupby(['user', 'movie'])['rating']
      .sum()
      .unstack()
    )
    df_list_binary.append(interactions)
'''

100%|██████████| 17770/17770 [02:27<00:00, 120.29it/s]


In [5]:
'''
pivot_df_binary = pd.DataFrame()
pivot_df_binary = pd.concat(df_list_binary, axis=1)
pivot_df_binary = pivot_df_binary.fillna(0)
pivot_df_binary
'''

movie,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#pivot_df_binary.to_parquet('ratings_df_binary.parquet')

In [8]:
sparse_matrix_binary = sparse.csr_matrix(pivot_df_binary.to_numpy())
del pivot_df_binary
train_test_binary = random_train_test_split(sparse_matrix_binary, test_percentage=0.2)
train_binary = train_test_binary[0]
test_binary = train_test_binary[1]

In [9]:
model = LightFM(loss='warp')
model.fit(train_binary, epochs=5, verbose=True)

Epoch: 100%|██████████| 5/5 [07:11<00:00, 86.37s/it]


In [10]:
import pickle

In [11]:
with open('model_binary.pickle', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
train_precision = precision_at_k(model, train_binary, k=5).mean()
test_precision = precision_at_k(model, train_binary, test_binary, k=5).mean()
train_auc = auc_score(model, train_binary).mean()
test_auc = auc_score(model, test_binary, train_interactions=train_binary).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.43, test 0.50.
AUC: train 0.96, test 0.96.


In [82]:
user = 5
result = model.predict(user, [x for x in range(1,17770)])
df_result = pd.DataFrame(result).sort_values(by=0, ascending=False)
recommendations = df_result.head(5)[0].index.values.tolist()
for i in recommendations:
    print(movie_titles[movie_titles['id'] == i]['name'].values[0])

The In-Laws
Shallow Grave
Ozzy Osbourne: Crown Prince of Darkness
Amityville 1992: It's About Time
Son of Paleface


In [81]:
for liked_movie in sparse_matrix_binary[user].indices[sparse_matrix_binary[user].data == 1]:
    print(movie_titles[movie_titles['id'] == liked_movie]['name'].values[0])

Peter Tosh: Stepping Razor: Red X
Bad Boy Bubby
Battle Queen 2020
Port of Shadows
The Other Side of Heaven
To Catch a Thief
Boz Scaggs: Greatest Hits Live
Devil Man
Morrissey: Oye Esteban!
Ozzy Osbourne: Live & Loud
Escape from New York
Alias: Season 2
The Job
Brides of Christ
Sherlock Holmes Faces Death
Evita
Sister My Sister
The Kung Fu Master
Dragons: Metal Ages: The Movie
Baa Baa Black Sheep: Season 1
Wilder Napalm
Law & Order: Criminal Intent: The First Year
Garfield and Friends: Vol. 3
Mariah Carey: #1's
Witness
North Shore
Hot Shot
Detroit 9000
Wild Strawberries
